In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import seaborn as sns
import matplotlib.pyplot as plt
import swifter
import multiprocessing
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
from collections import defaultdict
from dataclasses import dataclass
from torch import nn
import torch.nn.functional as F
import torch
import random
import copy

sns.set_style("darkgrid")

# Loading Processed Reviews

In [3]:
# Paths
PROCESSED_FOLDER = './data/processed/'
PROCESSED_REVIEWS_FILE = 'processed_reviews.csv'

In [4]:
reviews = pd.read_csv(os.path.join(PROCESSED_FOLDER, PROCESSED_REVIEWS_FILE))

In [5]:
reviews.head()

,review_id,user_id,item_id,text,rating,tokenized_text
0,255938,0,1,"First things first. My ""reviews"" system is exp...",8,"[['First', 'things', 'first', '.'], ['My', '``..."
1,259117,1,2,Let me start off by saying that Made in Abyss ...,10,"[['Let', 'me', 'start', 'off', 'by', 'saying',..."
2,253664,2,3,"Art 9/10: It is great, especially the actions ...",7,"[['Art', '9/10', ':', 'It', 'is', 'great', ','..."
3,247454,3,4,As someone who loves Studio Ghibli and its mov...,6,"[['As', 'someone', 'who', 'loves', 'Studio', '..."
4,23791,4,5,code geass is one of those series that everybo...,10,"[['code', 'geass', 'is', 'one', 'of', 'those',..."


# Converting Data for Modeling

In [6]:
# Convert item_id to 0 indexed
if min(reviews['item_id']) != 0:
    reviews['item_id'] = reviews['item_id'] - 1
    print("Done")

Done


In [7]:
@dataclass
class Review:
    user_id: int
    item_id: int
    rating: int

In [8]:
USER_KEY = 'user_id'
ITEM_KEY = 'item_id'
RATING_KEY = 'rating'

In [9]:
user_to_reviews = defaultdict(list)
for _, row in reviews.iterrows():
    user_id, item_id, rating = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY]
    user_to_reviews[user_id].append(Review(user_id, item_id, rating))

## Creating the score matrix

In [10]:
# users by items
X = np.zeros(shape=(reviews['user_id'].nunique(), reviews['item_id'].nunique()))

In [11]:
for _, row in reviews.iterrows():
    user_id, item_id, rating = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY]
    X[user_id][item_id] = rating

# Vanilla Matrix Factorization

In [12]:
def l2_regularization(values):
    return torch.sum(torch.square(values))

In [13]:
class VanillaMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20, regularization_constant=1e-6):
        super().__init__()
        self.user_factors = nn.Embedding(num_users, embedding_dim)
        self.item_factors = nn.Embedding(num_items, embedding_dim)
        self.regularization_constant = regularization_constant
        
    def forward(self, user: torch.LongTensor, item: torch.LongTensor):
        # (1,20) dot prod with (1,20) gives us (1,20)
        # Embedding output tensor has shape (*, n_factors)
        result_tensor = self.user_factors(user) * self.item_factors(item)
        # Sum along row
        pred_rating = torch.sum(result_tensor, axis=1)
        return pred_rating
    
    def loss(self, pred_rating, rating):
        # MSE
        MSE_loss = F.mse_loss(pred_rating, rating)
        
        # L2 Regularization
        sum_of_squared_values = l2_regularization(self.user_factors.weight) + l2_regularization(self.item_factors.weight)
        l2_penalty = self.regularization_constant * sum_of_squared_values
        
        # Total Loss
        total_loss = MSE_loss + l2_penalty
        return total_loss
    
    def predict(self, user_id: int, item_id: int):
        user = torch.LongTensor([user_id])
        item = torch.LongTensor([item_id])
        return self.forward(user, item)

## Train Function

In [44]:
def train(train_X, valid_X, model, optimizer, n_epochs=10):
    """Training Function, calculates training and validation loss"""
    
    loss_func = torch.nn.MSELoss()
    for epoch in range(1, n_epochs+1):
        rows, cols = X.nonzero()
        p = np.random.permutation(len(rows))
        rows, cols = rows[p], cols[p]
        
        total_MSE_loss = 0
        
        for row, col in zip(rows, cols):
            # Set gradients to zero
            optimizer.zero_grad()

            # Turn data into tensors
            rating = torch.FloatTensor([X[row, col]])
            row = torch.LongTensor([row])
            col = torch.LongTensor([col])

            # Predict and calculate loss
            pred_rating = model.forward(row, col)
            # assert pred_rating.shape == rating.shape
            loss = model.loss(pred_rating, rating)
            
            # Backpropagate
            loss.backward()

            # Update the parameters
            optimizer.step()
            
            # Total MSE Loss (just for status updates) for this epoch
            total_MSE_loss += F.mse_loss(pred_rating, rating)

        
        print(f"Epoch {epoch} MSE Loss: {total_MSE_loss / len(X.nonzero()[0])}")

## Train/Test Split

In [45]:
train_X = copy.deepcopy(X)
valid_X = np.zeros(shape=X.shape)
test_X = np.zeros(shape=X.shape)

for user_id, reviews in user_to_reviews.items():
    random.shuffle(reviews)
    # Leave one out for valid
    valid_review = reviews[0]
    train_X[valid_review.user_id][valid_review.item_id] = 0
    valid_X[valid_review.user_id][valid_review.item_id] = valid_review.rating
    
    # Leave one out for test
    test_review = reviews[1]
    train_X[test_review.user_id][test_review.item_id] = 0
    test_X[test_review.user_id][test_review.item_id] = test_review.rating
    
    # Rest for train

## Train Config

In [48]:
embedding_dim=20
lr=1e-2
regularization_constant=0

model = VanillaMF(num_users=X.shape[0], num_items=X.shape[1], embedding_dim=embedding_dim, regularization_constant=regularization_constant)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

## Training

In [ ]:
train(X, valid_X, model, optimizer)

Epoch 1 MSE Loss: 55.92216873168945
Epoch 2 MSE Loss: 7.399054527282715
Epoch 3 MSE Loss: 3.0362563133239746
Epoch 4 MSE Loss: 2.0175209045410156
Epoch 5 MSE Loss: 1.5612248182296753
